In [59]:
import random
import queue
import concurrent.futures
import math
from functools import reduce
from threading import Barrier

def share(secret, client_num):
    """
    Producing share based on the number of clients.
    """
    shares = [0] * client_num
    for i in range(client_num - 1):
        shares[i] = random.randint(1, 6)
    shares[client_num - 1] = secret - sum(shares[:client_num - 1])
    return shares

def A2B(A):
    B = []
    while A > 0:
        B.append(A % 2)
        A = math.floor(A / 2)
    B.reverse()
    return B

def B2A(B):
    print(B)
    return reduce(lambda a, b: 2*a + b, B)

def CP(i, x, y, U, V, W, input_queue, output_queue, barrier):
    """
    The main function each party executes separately.
    """
    
    def mul_number(i, s, d):
        """
        Multiplication function based on Beaver triple.
        """
        D = s - U
        E = d - V

        # Sending and receiving intermediate results
        for q in output_queue:
            q.put((D, E))

        DE_values = [(D, E)] + [input_queue.get() for _ in range(len(output_queue))]

        D_sum = sum(D for D, E in DE_values)
        E_sum = sum(E for D, E in DE_values)

        Z = W + (D_sum * V) + (U * E_sum)

        if i == 0:
            Z += (D_sum * E_sum)
        return Z
    
    def o(d2, d1):
        (p2, g2) = d2
        (p1, g1) = d1
        return (p1 & p2, g2 | (p2 & g1))
    
    def PreOpL(A):
        i = 1
        j = 2
        while j < len(A):
            for k in range(math.floor(len(A) / j)):
                p = i + k * j
                for q in range(i):
                    A[p+q] = o(A[p], A[p+q])
            i = j
            j *= 2
        return A
    
    def addbitwise(A, B):
        k = len(A)

        # Padding with zeroes for equal length
        if len(B) < k:
            B = [0] * (k - len(B)) + B
        elif k < len(B):
            A = [0] * (len(B) - k) + A
            k = len(B)
        
        D = [(a + b - 2*a*b, a*b) for (a, b) in zip(A, B)]
        C = PreOpL(D)
        S = [(a + b - 2*c) for (a, b, (_, c)) in zip(A, B, C)]
        S[:k-1] = [(s - c) for (s, (_, c)) in zip(S[:k-1], C[1:])]
        (_, c) = C[k-1]
        S = [c] + S
        return S

    return B2A(addbitwise(A2B(x), A2B(y)))

def SPDZ_prepare(beta1, beta2, group_size):
    """
    Function to produce Beaver triple shares.
    """
    U, V = random.randint(3, 6), random.randint(3, 6)
    W = U * V
    return share(U, group_size), share(V, group_size), share(W, group_size), share(beta1, group_size), share(beta2, group_size)

def SPDZ_execute(preparation_data, group_size):
    """
    Function to execute the SPDZ protocol.
    """
    with concurrent.futures.ThreadPoolExecutor() as executor:
        barrier = Barrier(group_size)
        U_shares, V_shares, W_shares, x_shares, y_shares = preparation_data
        queues = [queue.Queue() for _ in range(group_size)]
        futures = [
            executor.submit(
                CP, j, x_shares[j], y_shares[j], U_shares[j], V_shares[j], W_shares[j], 
                queues[j], [queues[k] for k in range(group_size) if k != j], barrier
            ) for j in range(group_size)
        ]

        group_results = [f.result() for f in futures]
        print("Final results share of each party:", group_results)
        sum_total = sum(group_results)
        print("Final Result:", sum_total)

# Main execution
if __name__ == "__main__":
    input1, input2 = 20, 10  # The multiplication input numbers
    client_num = 5  # Number of clients

    preparation_data = SPDZ_prepare(input1, input2, client_num)
    SPDZ_execute(preparation_data, client_num)


[0, 0, 0, 1][0, 0, 1, 0]

[0, -1, 0, 0]
[0, 1, 1, 1]


IndexError: list index out of range